In [116]:
import numpy as np
import pandas as pd
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_ALIGN_VERTICAL

In [117]:
# 定义函数
def docx_find_replace_text(doc, search_text, replace_text):
    for paragraph in doc.paragraphs:
        if search_text in paragraph.text:
            for run in paragraph.runs:
                if search_text in run.text:
                    run.text = run.text.replace(search_text, replace_text)

# docx_find_replace_text(doc, '«name»', 'new text')

# 替换编号
def docx_find_replace_header(doc, search_text, replace_text):
    for section in doc.sections:
        header = section.header  # 获取节的页眉
        for paragraph in header.paragraphs:  # 遍历页眉的每个段落
            if search_text in paragraph.text:
                for run in paragraph.runs:
                    if search_text in run.text:
                        run.text = run.text.replace(search_text, replace_text)

# docx_find_replace_header(doc, '«c_id»', 'new text')

#确定样式
def set_cell_text_style(cell, pos): 
    cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER # 垂直居中
    for para in cell.paragraphs:
        if pos.lower() == "left":
            para.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
        elif pos.lower() == "mid":
            para.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        elif pos.lower() == "right":
            para.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT
        else:
            raise ValueError("Invalid position value. Please use 'left', 'mid' or 'right'.")
        for run in para.runs:
            run.font.size = Pt(10.5) # 五号字体
            run.font.name = u"宋体"
            run._element.rPr.rFonts.set(qn('w:eastAsia'), u'宋体')
    return cell

In [118]:
df = pd.read_excel('../发出商品-函证-明细表.xlsx', sheet_name='发出商品明细大表')
df.shape

(2864, 6)

In [119]:
df.dropna(axis=0, inplace=True)
df['年份'] = df['年份'].map(int)

In [120]:
# # 测试模板
# doc = Document('./example_发出商品.docx')
# docx_find_replace_text(doc, '«name»', '这里是单位名称')
# docx_find_replace_header(doc, '«c_id»', '这里是函证编号')
# for i,table in enumerate(doc.tables):
#     print(f'Table {i} has {len(table.rows)} rows and {len(table.columns)} columns.')

In [121]:
comp = pd.read_excel('../发出商品-函证-明细表.xlsx', sheet_name='发出商品公司编号')
comp.dtypes

购货单位    object
函证编号    object
dtype: object

In [122]:
comp_name = comp['购货单位'].drop_duplicates().values
len(comp_name)

74

In [123]:
c1 = df['购货单位'].drop_duplicates().values
len(c1)

74

In [124]:
c1

array(['北京计算机技术及应用研究所', '西安工业大学', '中国船舶集团有限公司第七二四研究所',
       '中国电子科技集团公司第四十三研究所', '西安电子工程研究所', '北京无线电测量研究所', '中国长峰机电技术研究设计院',
       '西安微电子技术研究所', '上海航天电子通讯设备研究所', '贵州航天林泉电机有限公司', '天津津航计算技术研究所',
       '中国电子科技集团公司第五十五研究所', '中电科技（南京）电子信息发展有限公司',
       '中电科技（南京）电子信息发展有限公司雨花分公司', '武汉中元通信股份有限公司', '北京航天微电科技有限公司',
       '北京遥感设备研究所', '中国电子科技集团公司第二十六研究所', '洛阳隆盛科技有限责任公司', '南京长峰航天电子科技有限公司',
       '南京科瑞达电子装备有限责任公司', '合肥华耀电子工业有限公司', '中国电子科技集团公司第二十四研究所',
       '贵阳航空电机有限公司', '中国船舶集团有限公司第七一二研究所', '青岛航天半导体研究所有限公司',
       '中国船舶集团有限公司第七0九研究所', '中航物资装备有限公司', '深圳市科思科技股份有限公司', '南京国睿防务系统有限公司',
       '上海杰瑞兆新信息科技有限公司', '中电科蓉威电子技术有限公司', '成都天箭科技股份有限公司', '江苏无线电厂有限公司',
       '成都云上飞腾电子科技有限公司', '西安霍威电源有限公司', '成都四威功率电子科技有限公司', '成都智明达电子股份有限公司',
       '长沙湘计海盾科技有限公司', '上海瀚讯信息技术股份有限公司', '北京恒远盛宁科技有限公司', '成都宏明电子股份有限公司二厂',
       '武汉宝威德电子有限公司', '南京长江电子信息产业集团有限公司', '中国电子科技集团公司第五十四研究所',
       '四川合创益电子有限公司', '成都必控科技有限责任公司', 'Super Chip Ltd', '北京机械设备研究所',
       '陕西雷能电子科技有限公司', '北京飞航捷迅物资有限责任公司', '北京华航无线电测量研究所', '北京微科

In [125]:
df.dtypes

年份        int64
购货单位     object
函证编码     object
产品长代码    object
产品名称     object
实发数量      int64
dtype: object

In [126]:
# temp = df[(df['购货单位']=='北京计算机技术及应用研究所') & (df['年份']==2022)]
# temp = temp.groupby(['产品名称','产品长代码'])['实发数量'].sum()
# temp = pd.DataFrame(temp).reset_index(['产品长代码', '产品名称'])
# temp

In [127]:
df.columns

Index(['年份', '购货单位', '函证编码', '产品长代码', '产品名称', '实发数量'], dtype='object')

In [132]:
# 根据每个公司不同的票据数，生成不同的行数
for name in comp_name:
    doc = Document('./example_发出商品.docx')
    c_id = df[df['购货单位']==name]['函证编码'].drop_duplicates().values[0]
    docx_find_replace_text(doc, '«c_id»', c_id)
    docx_find_replace_text(doc, '«name»', name)
    table_num = 0
    
    for year in [2020,2021,2022,2023]:
        temp = df[(df['购货单位']==name) & (df['年份']==year)]
        temp = temp.groupby(['产品名称','产品长代码'])['实发数量'].sum()
        temp = pd.DataFrame(temp).reset_index(['产品长代码', '产品名称'])
        r_num = temp.shape[0]
        
        if r_num==0: # 当年无记录 跳到下一个表格进行判断
            table_num += 1
            
        else:   
            for i in range(r_num):
                adr = doc.tables[table_num].add_row()
                adr.cells[0].text = '发出商品' ; set_cell_text_style(adr.cells[0], 'mid')# 发出科目
                adr.cells[1].text = temp.iloc[i, 0].strip() ; set_cell_text_style(adr.cells[1], 'mid')# 产品名称
                adr.cells[2].text = temp.iloc[i, 1].strip() ; set_cell_text_style(adr.cells[2], 'mid')# 产品长代码
                adr.cells[3].text = "{:,.2f}".format(temp.iloc[i, 2]) ; set_cell_text_style(adr.cells[3], 'right')# 数量
                adr.cells[4].text = '否' ; set_cell_text_style(adr.cells[4], 'mid')# 是否有留置权
                adr.cells[5].text = '良好' ; set_cell_text_style(adr.cells[5], 'mid')# 状况
    
                
            # 循环结束后删除第二行
            row = doc.tables[table_num].rows[1]._element
            row.getparent().remove(row)
            # 发出商品不需要合计
            # # 循环结束后增加合计部分
            # row_cells = doc.tables[table_num].add_row().cells
            # merged_cell = row_cells[0].merge(row_cells[1]).merge(row_cells[2]).merge(row_cells[3]).merge(row_cells[4])\
            #     .merge(row_cells[5]).merge(row_cells[6])

            # # 将合并后的单元格的文本设置为"合计"
            # merged_cell.text = '合计:' ; set_cell_text_style(merged_cell, 'mid')
            # row_cells[7].text = "{:,.2f}".format(heji[name].get(year, '无')) ; set_cell_text_style(row_cells[7], 'right') # 合计金额
            # # # 获取单元格中的第一个段落
            # # para = merged_cell.paragraphs[0]

            # # # 设置段落的对齐方式为居中
            # # para.alignment = 'WD_PARAGRAPH_ALIGNMENT.CENTER'

            # # # 设置段落的字体为宋体，大小为5号
            # # run = para.runs[0]
            # # run.font.name = '宋体'
            # # run.font.size = Pt(21)  # 五号字对应21磅
            table_num += 1
                    
    doc.save(f'../3.2发出商品/{name}-发出商品询证函.docx')

